In [11]:
import pandas as pd
import numpy as np

In [12]:
# Loading the DataSets
all_08 = pd.read_csv('./DataSet/all_alpha_08_filtered.csv')
all_18 = pd.read_csv('./DataSet/all_alpha_18_filtered.csv')

In [13]:
all_08.shape, all_18.shape

((986, 13), (794, 13))

## Fixing Data Types
  
Making changes to make the datatypes consistent and practical to work with.

### Fix cyl datatype
  
- 2008: extract `int` from `string`.
- 2018: convert `float` to `int`.

In [14]:
# check value counts for the 2008 cyl column
all_08.cyl.value_counts()

(6 cyl)     409
(4 cyl)     283
(8 cyl)     199
(5 cyl)      48
(12 cyl)     30
(10 cyl)     14
(2 cyl)       2
(16 cyl)      1
Name: cyl, dtype: int64

In [15]:
# Extracting integers from string
all_08.cyl = all_08.cyl.str.extract('(\d+)').astype(int)

In [16]:
# Check if the extraction works correctly
all_08.cyl.value_counts()

6     409
4     283
8     199
5      48
12     30
10     14
2       2
16      1
Name: cyl, dtype: int64

In [17]:
# Changing float to int
all_18.cyl = all_18.cyl.astype(int)

In [18]:
# Check if the changing works
all_08.cyl.dtype, all_18.cyl.dtype

(dtype('int32'), dtype('int32'))

### Fix `air_pollution_score` datatype
  
- 2008: convert `string` to `float`.
- 2018: convert `int` to `float`.

In [19]:
# Check the unique values
all_08.air_pollution_score.unique()

array(['7', '6', '9.5', '9', '6/4'], dtype=object)

In [ ]:
# The code under won't work
all_08.air_pollution_score = all_08.air_pollution_score.astype(float)
all_08.air_pollution_score = pd.to_numeric(all_08.air_pollution_score)

**NOTE**
- Changing `air_pollution_score` using `astype` or `pd.to_numeric` won't work in this case.  
- The `6/4` restrict our ability to use the above function.  
- The `6/4` value represents a hybrid value (according to documentation)
  - This also affects the values for `city_mpg`, `hwy_mpg`, and `cmb_mp` columns
  - Their value also separated by `/` respective to the `fuel` type
  - So, we need to `explode` the values into two rows

In [21]:
all_08[all_08.city_mpg.str.contains('/')]

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
582,MERCEDES-BENZ C300,3.0,6,Auto-L7,2WD,ethanol/gas,small car,6/4,13/18,19/25,15/21,7/6,no


In [22]:
# Filter all `fuel`s with value containing `/`
hybrid_08 = all_08[all_08.fuel.str.contains('/')]
hybrid_18 = all_18[all_18.fuel.str.contains('/')]
hybrid_08.shape, hybrid_18.shape

((1, 13), (38, 13))

**Checking the datasets containing `/` values**

In [23]:
hybrid_08.head()

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
582,MERCEDES-BENZ C300,3.0,6,Auto-L7,2WD,ethanol/gas,small car,6/4,13/18,19/25,15/21,7/6,no


In [24]:
hybrid_18.head()

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
52,BMW 330e,2.0,4,SemiAuto-8,2WD,Gasoline/Electricity,small car,3,28/66,34/78,30/71,10,Yes
78,BMW 530e,2.0,4,SemiAuto-8,2WD,Gasoline/Electricity,small car,7,27/70,31/75,29/72,10,Elite
79,BMW 530e,2.0,4,SemiAuto-8,4WD,Gasoline/Electricity,small car,7,27/66,31/68,28/67,10,Elite
92,BMW 740e,2.0,4,SemiAuto-8,4WD,Gasoline/Electricity,large car,3,25/62,29/68,27/64,9,Yes
189,CHEVROLET Impala,3.6,6,SemiAuto-6,2WD,Ethanol/Gas,large car,5,14/18,20/28,16/22,4,No


**Expanding the values from `str` to `list`**
  
It seems in `2008` the `air_pollution_score` and `greenhouse_gas_score` values are also affected by `fuel` type  
We add the above listed columns to expand their values as well

In [25]:
expanded_08 = hybrid_08.assign(
    fuel=hybrid_08.fuel.str.split('/'),
    air_pollution_score=hybrid_08.air_pollution_score.str.split('/'),
    city_mpg=hybrid_08.city_mpg.str.split('/'),
    hwy_mpg=hybrid_08.hwy_mpg.str.split('/'),
    cmb_mpg=hybrid_08.cmb_mpg.str.split('/'),
    greenhouse_gas_score=hybrid_08.greenhouse_gas_score.str.split('/'),
)
expanded_18 = hybrid_18.assign(
    fuel=hybrid_18.fuel.str.split('/'),
    city_mpg=hybrid_18.city_mpg.str.split('/'),
    hwy_mpg=hybrid_18.hwy_mpg.str.split('/'),
    cmb_mpg=hybrid_18.cmb_mpg.str.split('/'),
)

In [26]:
# Check if expansion works
expanded_08.head()

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
582,MERCEDES-BENZ C300,3.0,6,Auto-L7,2WD,"[ethanol, gas]",small car,"[6, 4]","[13, 18]","[19, 25]","[15, 21]","[7, 6]",no


In [27]:
# Check if expansion works
expanded_18.head()

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
52,BMW 330e,2.0,4,SemiAuto-8,2WD,"[Gasoline, Electricity]",small car,3,"[28, 66]","[34, 78]","[30, 71]",10,Yes
78,BMW 530e,2.0,4,SemiAuto-8,2WD,"[Gasoline, Electricity]",small car,7,"[27, 70]","[31, 75]","[29, 72]",10,Elite
79,BMW 530e,2.0,4,SemiAuto-8,4WD,"[Gasoline, Electricity]",small car,7,"[27, 66]","[31, 68]","[28, 67]",10,Elite
92,BMW 740e,2.0,4,SemiAuto-8,4WD,"[Gasoline, Electricity]",large car,3,"[25, 62]","[29, 68]","[27, 64]",9,Yes
189,CHEVROLET Impala,3.6,6,SemiAuto-6,2WD,"[Ethanol, Gas]",large car,5,"[14, 18]","[20, 28]","[16, 22]",4,No


**Explode the expanded values to rows**  
We need to use the exact number of columns we used to expand the values

In [28]:
fixed_08 = expanded_08.explode([
    'fuel',
    'air_pollution_score',
    'city_mpg',
    'hwy_mpg',
    'cmb_mpg',
    'greenhouse_gas_score',
])
fixed_18 = expanded_18.explode([
    'fuel',
    'city_mpg',
    'hwy_mpg',
    'cmb_mpg',
])

In [29]:
# Check if it is expanding works
fixed_08.head()

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
582,MERCEDES-BENZ C300,3.0,6,Auto-L7,2WD,ethanol,small car,6,13,19,15,7,no
582,MERCEDES-BENZ C300,3.0,6,Auto-L7,2WD,gas,small car,4,18,25,21,6,no


In [30]:
# Check if it is expanding works
fixed_18.head()

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
52,BMW 330e,2.0,4,SemiAuto-8,2WD,Gasoline,small car,3,28,34,30,10,Yes
52,BMW 330e,2.0,4,SemiAuto-8,2WD,Electricity,small car,3,66,78,71,10,Yes
78,BMW 530e,2.0,4,SemiAuto-8,2WD,Gasoline,small car,7,27,31,29,10,Elite
78,BMW 530e,2.0,4,SemiAuto-8,2WD,Electricity,small car,7,70,75,72,10,Elite
79,BMW 530e,2.0,4,SemiAuto-8,4WD,Gasoline,small car,7,27,31,28,10,Elite


In [31]:
# Checking the shape of exploded data points
fixed_08.shape, fixed_18.shape

((2, 13), (76, 13))

#### Dropping and Merging
  
We need to remove the invalid ones  
Merge the fixed datasets and original datasets  

In [32]:
expanded_08 = all_08.copy()
expanded_18 = all_18.copy()

In [33]:
# drop the original hybrid rows
all_08.drop(hybrid_08.index, inplace=True)
all_18.drop(hybrid_18.index, inplace=True)

# Check the shape
all_08.shape, all_18.shape

((985, 13), (756, 13))

In [34]:
# Merging in our newly separated rows
all_08 = pd.concat([all_08, fixed_08], ignore_index=True)
all_18 = pd.concat([all_18, fixed_18], ignore_index=True)

# Free up the space
del fixed_08, fixed_18

# Check the shape
all_08.shape, all_18.shape

((987, 13), (832, 13))

#### Changing the Datatype
  
Now we can change the datatype using either `astype` or `pd.to_numeric`

In [37]:
# Check if the hybrid values are fixed
all_08.air_pollution_score.unique(), all_18.air_pollution_score.unique()

(array(['7', '6', '9.5', '9', '4'], dtype=object),
 array([3, 1, 7, 5, 6], dtype=int64))

In [38]:
# Changing the datatypes to float
all_08.air_pollution_score = all_08.air_pollution_score.astype(float)
all_18.air_pollution_score = all_18.air_pollution_score.astype(float)

### Fix `city_mpg`, `hwy_mpg`, `cmb_mpg` datatypes
  
- 2008 and 2018: convert `string` to `float`.
  
Fixing the values for `air_pollution_score` makes these conversion easier

In [46]:
# Check if there is inaproppriet values
for col in ['city_mpg', 'hwy_mpg', 'cmb_mpg']:
    print(f"{col}_08:", all_08[col].apply(lambda x: "/" in x).any())
    print(f"{col}_18:", all_18[col].apply(lambda x: "/" in x).any())
    print()

city_mpg_08: False
city_mpg_18: False

hwy_mpg_08: False
hwy_mpg_18: False

cmb_mpg_08: False
cmb_mpg_18: False



In [52]:
# Changing the Datatypes
all_08[['city_mpg', 'hwy_mpg', 'cmb_mpg']] = all_08[['city_mpg', 'hwy_mpg', 'cmb_mpg']].astype(float)
all_18[['city_mpg', 'hwy_mpg', 'cmb_mpg']] = all_18[['city_mpg', 'hwy_mpg', 'cmb_mpg']].astype(float)

### Fix `greenhouse_gas_score` datatype
  
- 2008: convert from `float` to `int`.

In [53]:
all_08.greenhouse_gas_score = all_08.greenhouse_gas_score.astype(int)
all_18.greenhouse_gas_score = all_18.greenhouse_gas_score.astype(int)

In [54]:
# Check the datatypes
all_08.dtypes

model                    object
displ                   float64
cyl                       int32
trans                    object
drive                    object
fuel                     object
veh_class                object
air_pollution_score     float64
city_mpg                float64
hwy_mpg                 float64
cmb_mpg                 float64
greenhouse_gas_score      int32
smartway                 object
dtype: object

In [55]:
all_18.dtypes

model                    object
displ                   float64
cyl                       int32
trans                    object
drive                    object
fuel                     object
veh_class                object
air_pollution_score     float64
city_mpg                float64
hwy_mpg                 float64
cmb_mpg                 float64
greenhouse_gas_score      int32
smartway                 object
dtype: object

In [59]:
# Check if the two dataframe column types are equal
all_08.dtypes == all_18.dtypes

model                   True
displ                   True
cyl                     True
trans                   True
drive                   True
fuel                    True
veh_class               True
air_pollution_score     True
city_mpg                True
hwy_mpg                 True
cmb_mpg                 True
greenhouse_gas_score    True
smartway                True
dtype: bool

## Storing the Final result

In [57]:
all_08.to_csv('./DataSet/all_alpha_08_fixed_datatypes.csv', index=False)
all_18.to_csv('./DataSet/all_alpha_18_fixed_datatypes.csv', index=False)